In [48]:
import pandas as pd
import math

# splitting
from sklearn.model_selection import train_test_split

# metrics
from sklearn.metrics import mean_squared_error

# GBM model
from sklearn.ensemble import GradientBoostingRegressor

In [49]:
len(all_scores)

2462

In [13]:
all_scores = pd.read_csv('data/all_scores_by_season_edited.csv')
draft = pd.read_csv('data/drafts/all_drafts.csv')
merged = pd.merge(all_scores, draft, on="twitter_handle")
merged.rename(columns={'Year':'Draft_Year'}, inplace=True)
# Q: filling all the empty scores with 0s since they never played
merged = merged.fillna(0)

In [18]:
a = [int(m[5:]) for m in merged['season']] <= merged['Draft_Year']
predraft = merged[a]
predraft = predraft.dropna()
predraft.sample(3)

,index,twitter_handle,season,cleaned_text,doc_labels,doc_labels_weights,emot_anger,emot_disgust,emot_fear,emot_joy,...,FT%,MP.1,PTS.1,TRB.1,AST.1,WS,WS/48,BPM,VORP,Draft_Year
89,97,denzelvalentine,2015-2016,: denzel valentine wearing a familiar jersey a...,/sports/basketball,0.884262,0.065457,0.092857,0.062506,0.710699,...,0.759,20.8,7.7,3.7,2.0,3.8,0.049,-1.6,0.4,2016
366,394,PAPPYGAWD,2012-2013,": , we just out here trynna eat .. rt : i coac...",/sports/basketball /sports/go kart /art and en...,0.828274 0.752314 0.741071 0.675489 0.669319 0...,0.067120,0.113392,0.052143,0.640327,...,0.857,10.5,4.4,1.1,2.4,0.0,0.021,-4.4,-0.1,2013
977,1056,thats_G_,2012-2013,yoooo this dude is a fool!! straight comedy rt...,/sports/go kart /art and entertainment/shows a...,0.739053 0.73413 0.651626 0.635493 0.613623 0....,0.095812,0.096346,0.089631,0.689541,...,0.801,29.1,12.1,2.6,2.1,18.6,0.080,-1.0,2.7,2014


In [15]:
print('We have data on '+str(len(predraft))+' unique pre-draft seasons.')
print('That\'s '+str(round(len(predraft)/len(all_scores)*100,2))+'% of the gathered IBM scores.')


# What percentage of players HAVE a pre-draft season?
# Do they even have pre-draft? (better players more likely to tweet, maybe)
#     Frequency of tweets

# Of the all-in-one metrics
#    BPM, BPM 2.0 most trustworthy
#    OBPM, DBPM
#    AST%, STL%, etc
#    Scouting/Public Rating? (publicly difficult, draft boards)

# Scouting will help with the other draft model features
#    bio stuff


# Outlier stuff with whether players change in personality over time
#     The people who do change, why do they change?

# Defensive player communication is important
#   maybe that publicly comes out in their tweets
#   usage% by tweet personality



# How different is a professional basketball player's tweet ensemble from a normal person/other athletes?
# On-court vs off-court personality - does his personality change over time


# Technical foul% by IBM scores
#   is there a specific type of personality?

We have data on 579 unique pre-draft seasons.
That's 23.52% of the gathered IBM scores.


In [38]:
measure_cols = ['emot_anger','emot_disgust','emot_fear','emot_joy','emot_sadness','sent_score','need_challenge', 'need_closeness', 'need_curiosity',
       'need_excitement', 'need_harmony', 'need_ideal', 'need_liberty',
       'need_love', 'need_practicality', 'need_self_expression',
       'need_stability', 'need_structure', 'facet_adventurousness',
       'facet_artistic_interests', 'facet_emotionality', 'facet_imagination',
       'facet_intellect', 'facet_liberalism', 'facet_achievement_striving',
       'facet_cautiousness', 'facet_dutifulness', 'facet_orderliness',
       'facet_self_discipline', 'facet_self_efficacy', 'facet_activity_level',
       'facet_assertiveness', 'facet_cheerfulness', 'facet_excitement_seeking',
       'facet_friendliness', 'facet_gregariousness', 'facet_altruism',
       'facet_cooperation', 'facet_modesty', 'facet_morality',
       'facet_sympathy', 'facet_trust', 'facet_anger', 'facet_anxiety',
       'facet_depression', 'facet_immoderation', 'facet_self_consciousness',
       'facet_vulnerability', 'value_conservation', 'value_openness_to_change',
       'value_hedonism', 'value_self_enhancement', 'value_self_transcendence']
target_cols = ['Yrs','WS','WS/48','BPM','VORP']

In [45]:
def models_all_targets(predraft, measure_cols, target_cols, vb = 0):
    X = predraft[measure_cols]
    models = []
    for t in target_cols:
        y = predraft[t]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
        gbr = GradientBoostingRegressor(loss='huber',
                               learning_rate=0.1,
                               n_estimators=100,
                               subsample=1.0,
                               criterion='friedman_mse',
                               min_samples_split=2,
                               min_samples_leaf=1,
                               min_weight_fraction_leaf=0.0,
                               max_depth=5,
                               min_impurity_decrease=0.0,
                               min_impurity_split=None,
                               random_state=42,
                               alpha=0.9,
                               verbose=vb,
                               max_leaf_nodes=None,
                               warm_start=False,
                               validation_fraction=0.1,
                               n_iter_no_change=None,
                               tol=0.0001,
                               ccp_alpha=0.0)
        print('Now fitting model on target '+t+'...')
        gbr.fit(X_train, y_train)
        test_preds = gbr.predict(X_test)
        test_mse = mean_squared_error(y_test, test_preds)
        print('MSE for '+t+' model: '+str(test_mse))
        print('RMSE for '+t+' model: '+str(math.sqrt(test_mse)))
        test_score = gbr.score(X_test, y_test)
        print('R^2 for '+t+'model: '+str(test_score))
        models.append(gbr)
        print('\n')
    
    return models

In [47]:
models = models_all_targets(predraft, measure_cols, target_cols)

Now fitting model on target Yrs...
MSE for Yrs model: 4.112597999474088
RMSE for Yrs model: 2.027954141363677
R^2 for Yrsmodel: -0.07326779785090864


Now fitting model on target WS...
MSE for WS model: 91.98468800441215
RMSE for WS model: 9.590864820463906
R^2 for WSmodel: -0.1422168298264861


Now fitting model on target WS/48...
MSE for WS/48 model: 0.006942187447649595
RMSE for WS/48 model: 0.08331979025207394
R^2 for WS/48model: -0.0443469960660714


Now fitting model on target BPM...
MSE for BPM model: 11.995382419625326
RMSE for BPM model: 3.4634350606912387
R^2 for BPMmodel: 0.12283450548092524


Now fitting model on target VORP...
MSE for VORP model: 10.362877039050296
RMSE for VORP model: 3.2191422831323093
R^2 for VORPmodel: -0.25718523106778646




### TODO:
- Check data (need API key)
- 